In [160]:
from datasets import load_dataset
import torch
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
from transformers import BertTokenizerFast, TFBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

In [161]:
dataset = load_dataset("commonsense_qa")

Found cached dataset parquet (C:/Users/Mia/.cache/huggingface/datasets/parquet/commonsense_qa-4d2bfefd7b5e8ac5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [162]:
generics_kb = load_dataset("generics_kb", "generics_kb")

Found cached dataset generics_kb (C:/Users/Mia/.cache/huggingface/datasets/generics_kb/generics_kb/1.0.0/9b41cde494db24f842a9260588bcfb2e3a257364568666ef240e98c70fb0e709)


  0%|          | 0/1 [00:00<?, ?it/s]

In [171]:
train_ds = dataset['train'][:5000]
test_ds = dataset['test']
val_ds = dataset['validation']

In [186]:
questions = train_ds['question']
choices = [choice['text'] for choice in train_ds['choices']]
answers = train_ds['answerKey']

In [187]:
kb_sentences = generics_kb['train']['generic_sentence']

In [188]:
# decrease the size of the knowledge base to speed up encoding
kb_sentences = random.sample(kb_sentences, 1000)

In [189]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [190]:
len(kb_sentences)

1000

In [191]:
max_len = 27

In [192]:
knowledge_base_tokens = [tokenizer.encode(sentence, return_tensors='pt')[0] for sentence in kb_sentences]

In [193]:
knowledge_base_tokens_padded = torch.stack([
    torch.nn.functional.pad(token, (0, max_len - len(token)))
    for token in knowledge_base_tokens
])

In [194]:
def convert_answer_for_index(i, ds):
    answer_index = ord(ds['answerKey'][i]) - ord('A')
    return ds['choices'][i]['text'][answer_index] 

Tried different methods for semantic similarity, but found that cosine similarity works just as well but faster in this case. Because I am working with a smaller dataset of the knowledge base, the sentences dont have a big semantic similarity for humans. I tried:
- spaCy's similarity method
- cosine similarity from sentence transformers, util package 

In [195]:
sentences_merged = []
# counter = 0
for i in range(len(questions)):
    question = questions[i]
    choices_i = choices[i]

    question_tokens = tokenizer.encode(question, return_tensors='pt')[0]
    question_tokens = torch.nn.functional.pad(question_tokens, (0, max_len - len(question_tokens)))
    # print("Question Tokens Shape:", question_tokens.shape)
    # print("Knowledge Base Tokens Shape:", knowledge_base_tokens_padded[0].shape)
    similarities = [cosine_similarity(question_tokens.detach().reshape(1, -1).numpy(), kb_token.reshape(1, -1).detach().numpy())[0][0] for kb_token in knowledge_base_tokens_padded]

    most_similar_indexes = np.argsort(similarities)[-3:]
    most_similar_indexes = [int(x) for x in most_similar_indexes]

    input_text = f"{question}"+" ".join([kb_sentences[i] for i in most_similar_indexes])
    print(input_text)

    sentences_merged.append(input_text)
    # counter += 1
    # print(counter)

The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?Electronic interfaces are important if the encoder is to work properly with external controls. Frequency modulation is the rate of phase modulation. Academic counselors work with students to enhance their university experience and academic success.
Sammy wanted to go to where the people were.  Where might he go?Utmost care is taken to keep the lab and the lab equipments clean. Inflammation is a natural response of the body to any disease or damage. Progression can stop after a number of years, or can continue throughout life.
To locate a choker not located in a jewelry box or boutique where would you go?Content includes the elements of case management process, as well as ethical and legal issues. Publishing owns and operates a complete broadcast quality digital audio and video editing studio. Risk management is embedded in nearly every facet of planning and implemen

In [196]:
len(sentences_merged)

5000

In [197]:
sequences = []
answers_converted = [convert_answer_for_index(i, train_ds) for i in range(0, len(answers))]
for sentence, answer in zip(sentences_merged, answers_converted):
        sequences.append(f'{sentence} - {answer}')

In [198]:
len(sequences)

5000

In [199]:
tokenized_data = tokenizer(
    sequences,
    padding='max_length',  
    return_tensors='tf',
    truncation=True,
    max_length=100 
)

In [200]:
len(tokenized_data['input_ids'])

5000

In [201]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [202]:
input_ids = Input(shape=(100,), name='input_token', dtype='int32')
att_masks = Input(shape=(100,), name='masked_token', dtype='int32')
bert_in = bert_model(input_ids, attention_mask=att_masks)[1]
answer_output = Dense(5, activation='softmax', name='answer')(bert_in)

In [203]:
from tensorflow.keras.losses import mean_squared_error

In [204]:
model = Model(inputs=[input_ids, att_masks], outputs=[answer_output])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [205]:
print(len(tokenized_data['input_ids']))
print(len(tokenized_data['attention_mask']))
print(len(answers))

5000
5000
5000


In [206]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [207]:
answer_data = np.array(answers).reshape(-1, 1)
encoder = OneHotEncoder(categories=[['A', 'B', 'C', 'D', 'E']])
encoded_answer_data = encoder.fit_transform(answer_data).toarray()

In [208]:
encoded_answer_data.shape

(5000, 5)

In [209]:
model.fit([tokenized_data['input_ids'], tokenized_data['attention_mask']], encoded_answer_data, epochs=5, batch_size=32)

Epoch 1/5
152/157 [============================>.] - ETA: 55s - loss: 1.7020 - accuracy: 0.1916 

In [157]:
pred_tokens = tokenizer(val_ds['question'], max_length=100, return_tensors='np', truncation=True, pad_to_max_length=True)

In [158]:
pred_tokens['attention_mask'].shape

(1221, 100)

In [159]:
pred = model.predict([pred_tokens['input_ids'], pred_tokens['attention_mask']])

17/39 [============>.................] - ETA: 1:19

KeyboardInterrupt: 

In [ ]:
y_pred = []
answerKeys = ['A', 'B', 'C', 'D', 'E']

for p in pred:
    y_pred.append(answerKeys[np.argmax(p)])

In [ ]:
y_pred

['B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B'

In [138]:
y_test = val_ds['answerKey']

In [139]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [142]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [106]:
y_pred

['D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D'

In [107]:
y_test

['A',
 'A',
 'B',
 'A',
 'A',
 'C',
 'B',
 'D',
 'A',
 'C',
 'E',
 'D',
 'A',
 'D',
 'C',
 'D',
 'D',
 'E',
 'E',
 'D',
 'D',
 'C',
 'D',
 'D',
 'E',
 'D',
 'B',
 'A',
 'B',
 'B',
 'B',
 'A',
 'E',
 'D',
 'E',
 'C',
 'C',
 'B',
 'A',
 'B',
 'B',
 'B',
 'E',
 'E',
 'D',
 'E',
 'A',
 'E',
 'E',
 'C',
 'C',
 'B',
 'D',
 'C',
 'D',
 'E',
 'B',
 'D',
 'B',
 'A',
 'B',
 'E',
 'C',
 'D',
 'A',
 'A',
 'D',
 'A',
 'C',
 'D',
 'E',
 'E',
 'D',
 'A',
 'C',
 'B',
 'C',
 'E',
 'B',
 'C',
 'B',
 'D',
 'E',
 'A',
 'E',
 'A',
 'D',
 'B',
 'E',
 'C',
 'C',
 'C',
 'D',
 'E',
 'E',
 'E',
 'B',
 'D',
 'B',
 'E',
 'B',
 'D',
 'D',
 'E',
 'D',
 'D',
 'E',
 'A',
 'A',
 'E',
 'C',
 'C',
 'E',
 'D',
 'B',
 'B',
 'E',
 'C',
 'E',
 'C',
 'B',
 'C',
 'D',
 'D',
 'A',
 'B',
 'D',
 'B',
 'E',
 'D',
 'D',
 'C',
 'A',
 'D',
 'D',
 'B',
 'B',
 'E',
 'E',
 'A',
 'D',
 'E',
 'B',
 'A',
 'D',
 'E',
 'C',
 'C',
 'C',
 'E',
 'A',
 'B',
 'D',
 'D',
 'E',
 'D',
 'B',
 'E',
 'A',
 'E',
 'A',
 'D',
 'C',
 'B',
 'C',
 'A',
 'E'